In [1]:
import cv2
import os
import numpy as np
import time
from os import walk
import pickle

In [2]:
net = cv2.dnn.readNet("yolov3-tiny.weights","yolov3-tiny.cfg") #Tiny Yolo
classes = ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [34]:
imageFileNames = []
for (dirpath, dirnames, filenames) in walk("imageDir"):
    imageFileNames.extend(filenames)
    break
COLORS= np.random.uniform(0,255,size=(len(classes),3))
def crop_img_and_save(img, class_id, x, y, x_plus_w, y_plus_h, filename_to_save):
    if str(classes[class_id]) == "car":
        height,width,channels = img.shape
        x,y,x_plus_w, y_plus_h = max(0,x), max(0,y), max(0,x_plus_w), max(0,y_plus_h)
        x,y,x_plus_w, y_plus_h = min(360,x), min(288,y), min(360,x_plus_w), min(288,y_plus_h)
#         print(height,width,channels, x, y, x_plus_w, y_plus_h)
        crop_img = img[y:y_plus_h, x:x_plus_w]
        cv2.imwrite(filename_to_save, crop_img)
def draw_bounding_box_and_save(img, class_id, confidence, x, y, x_plus_w, y_plus_h, filename_to_save):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    cv2.imwrite(filename_to_save, img)

In [4]:
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

font = cv2.FONT_HERSHEY_PLAIN
output = {}
for fileName in imageFileNames:
    imageFile = 'imageDir/' + fileName
    # imageFile = 'imageDir/image1.jpg'
    img = cv2.imread(imageFile)
    # img = cv2.resize(img, (360,360))
    height,width,channels = img.shape
    blob = cv2.dnn.blobFromImage(img,(1.0/255.0),(416,416),(0,0,0),True,crop=False) 
    net.setInput(blob)
    outs = net.forward(outputlayers)
    class_ids=[]
    confidences=[]
    boxes=[]
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                #object detected
                center_x= int(detection[0]*width)
                center_y= int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x=int(center_x - w/2)
                y=int(center_y - h/2)

                boxes.append([x,y,w,h]) #put all rectangle areas
                confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
                class_ids.append(class_id) #name of the object tha was detected
    output[fileName] = (class_ids, confidences, boxes)
output

{'image1.jpg': ([2], [0.8962223529815674], [[213, 116, 145, 66]]),
 'image10.jpg': ([2], [0.9048565626144409], [[78, 117, 150, 65]]),
 'image100.jpg': ([2, 2],
  [0.9137676358222961, 0.8177433609962463],
  [[122, 132, 132, 48], [111, 132, 150, 47]]),
 'image1000.jpg': ([], [], []),
 'image1001.jpg': ([], [], []),
 'image1002.jpg': ([], [], []),
 'image1003.jpg': ([], [], []),
 'image1004.jpg': ([], [], []),
 'image1005.jpg': ([], [], []),
 'image1006.jpg': ([], [], []),
 'image1007.jpg': ([], [], []),
 'image1008.jpg': ([], [], []),
 'image1009.jpg': ([], [], []),
 'image101.jpg': ([2, 2],
  [0.6611438393592834, 0.6896518468856812],
  [[101, 123, 139, 57], [91, 131, 155, 49]]),
 'image1010.jpg': ([], [], []),
 'image1011.jpg': ([], [], []),
 'image1012.jpg': ([], [], []),
 'image1013.jpg': ([], [], []),
 'image1014.jpg': ([], [], []),
 'image1015.jpg': ([], [], []),
 'image1016.jpg': ([], [], []),
 'image1017.jpg': ([], [], []),
 'image1018.jpg': ([], [], []),
 'image1019.jpg': ([], []

In [5]:
pickle.dump(output,open("TinyYolo.pkl",'wb'))

In [35]:
yoloDirName = 'TinyYoloOutput'
cropDirName = 'CropImageOutput'
try:
    os.mkdir(yoloDirName)
    print("Directory" , yoloDirName ,  "Created ") 
except FileExistsError:
    print("Directory" , yoloDirName ,  "already exists")
try:
    os.mkdir(cropDirName)
    print("Directory" , cropDirName ,  "Created ") 
except FileExistsError:
    print("Directory" , cropDirName ,  "already exists")

Directory TinyYoloOutput Created 
Directory CropImageOutput Created 


In [36]:
for filename in output:
    imageFile = 'imageDir/' + filename
    img = cv2.imread(imageFile)
    class_ids, confidences, boxes = output[filename]
#     print(class_ids, confidences, boxes,filename)
    # apply non-max suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    count = 0 
    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        cropFilename = cropDirName+"/"+filename
        yoloFilename = yoloDirName+"/"+filename
        draw_bounding_box_and_save(img, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h), yoloFilename)
        crop_img_and_save(img, class_ids[i], round(x), round(y), round(x+w), round(y+h), cropFilename)

# release resources
cv2.destroyAllWindows()

288 360 3 213 116 358 182
288 360 3 78 117 228 182
288 360 3 122 132 254 180
288 360 3 91 131 246 180
288 360 3 86 133 228 179
288 360 3 52 131 227 181
288 360 3 47 132 197 179
288 360 3 298 120 360 180
288 360 3 289 114 359 181
288 360 3 270 109 360 181
288 360 3 260 115 359 182
288 360 3 236 110 360 182
288 360 3 217 115 360 181
288 360 3 193 134 360 184
288 360 3 189 114 345 182
288 360 3 178 114 331 180
288 360 3 28 114 181 180
288 360 3 158 118 317 180
288 360 3 136 119 291 180
288 360 3 123 117 279 180
288 360 3 102 117 260 180
288 360 3 92 113 248 180
288 360 3 70 115 230 183
288 360 3 49 116 211 181
288 360 3 37 111 197 180
288 360 3 21 132 181 181
288 360 3 1 114 157 182
288 360 3 22 123 159 181
288 360 3 0 130 151 183
288 360 3 0 110 134 180
288 360 3 2 107 113 180
288 360 3 0 115 97 180
288 360 3 0 110 77 179
288 360 3 0 105 63 181
288 360 3 0 94 45 180
288 360 3 7 122 149 181
288 360 3 0 121 125 180
288 360 3 0 121 119 181
288 360 3 59 118 209 183
288 360 3 0 123 100 175
28

288 360 3 0 114 133 164
288 360 3 95 191 278 260
288 360 3 0 109 123 166
288 360 3 117 205 292 255
288 360 3 0 111 106 167
288 360 3 0 112 84 166
288 360 3 0 105 71 169
288 360 3 177 202 348 253
288 360 3 200 195 358 264
288 360 3 216 204 359 251
288 360 3 172 131 329 182
288 360 3 229 162 357 256
288 360 3 254 197 360 256
288 360 3 275 200 359 256
288 360 3 294 195 355 256
288 360 3 165 134 305 179
288 360 3 130 133 305 181
288 360 3 126 131 279 180
